In [28]:
import pandas as pd
import numpy as np
import datetime
from datetime import date, time
import calendar

class Agenda:

   def __init__(self, nombre, datos = pd.DataFrame([])):
      self.nombre = nombre
      self.csv = nombre+'.csv'
      self.__guardadEnCsv(datos)

   def __guardadEnCsv(self, datos):
      datos.to_csv(self.csv)

   def leerDatos(self):
      return pd.read_csv(self.csv, index_col=0)
   
   def mostrarDatos(self):
      print(self.leerDatos())
   
   def insertarDatos(self, datos):
      df = self.leerDatos()
      df = pd.concat([df, datos], axis=0, ignore_index=True)
      self.__guardadEnCsv(df)
   
   def columnasDias(self):
      df = self.leerDatos()
      df = df.columns.values[3:]
      return list(df)   

   def guardarTurno(self, id_espe, paciente, dia, horario):
      df = self.leerDatos()
      df.loc[(df['id_espe']==id_espe) & (df['Horario']==horario), dia] = paciente
      print(df)  

class Especialidad:
   diasDeLaSemana = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes']
   horaApertura = '10:00'
   horaCierre = '17:00'

   def __init__(self, nombre, id, medico, duracionTurno):
      self.nombre = nombre
      self.id = id
      self.medico = medico
      self.duracionTurno = duracionTurno
      self.__dias = []

   def establecerDias(self, diaInt = 0):
      if diaInt == 0:
         print("Debe establecer los dias de atencion")
         for i in range(len(self.diasDeLaSemana)):
            print (i+1,'-', self.diasDeLaSemana[i])
         print("7 - Lunes a viernes")
         print("8 - TODOS")
         print("0 - SALIR")

         diaInt = int(input("Por favor seleccione los días de atencion de la especialidad, o 0 (cero) para salir"))

      elif (diaInt == 7):
         dias = list(range(0,5))
      elif (diaInt == 8):
         dias = list(range(0,6))
      elif (isinstance(diaInt, list)):
         dias = [x-1 for x in diaInt]
      else:
         dias = []
         while diaInt != 0:
            dias.append(diaInt-1)
            diaInt = int(input("Por favor seleccione los días de atencion de la especialidad, o 0 (cero) para salir"))

      self.__dias = dias
      
   def __formatSemana(self, anio, mes, semana):
      calen = calendar.Calendar()
      semana = calen.monthdayscalendar(anio, mes)[semana]
      diaDelaSemana = self.diasDeLaSemana
      dias = []
      for i in self.__dias:
         dias.append(diaDelaSemana[i]+' '+str(semana[i])+'/'+str(mes))
      return dias

   def establecerTurnos(self, mes, semana, dias = 0):
      anio = 2021

      if (len(self.__dias) == 0):
         self.establecerDias(dias)

      self.diasDisponibles = self.__formatSemana(anio, mes, semana)
      
      if not len(self.diasDisponibles) == 0:
         encabezados = ['id_espe','Especialidad','Horario']+self.diasDisponibles
         inicio = datetime.datetime.strptime(self.horaApertura, '%H:%M')
         turno = datetime.datetime.strptime(self.duracionTurno, '%H:%M')
         time_zero = datetime.datetime.strptime('00:00', '%H:%M')
         final = datetime.datetime.strptime(self.horaCierre, '%H:%M')
         
         final = final - turno + time_zero
         
         filas = []
         while inicio <= final:
            fila = [self.id, self.nombre, inicio.time()]+['disponible' for x in range(len(self.diasDisponibles))]
            inicio = inicio - time_zero + turno
            filas.append(fila)

         df = pd.DataFrame(filas, columns=encabezados)
         #print(df)

      self.turnero.insertarDatos(df)

   def traerDisponibles(self):
      df = self.turnero.leerDatos()
      df = df[df['id_espe'].eq(self.id)]
      df = df.loc[:, ['Horario']+self.diasDisponibles]
      return df

   def asignarTurno(self,paciente):
      disponibles = self.traerDisponibles()
      dias = disponibles.columns.values[1:]
      for i in range(len(dias)):
         print (str(i+1),'-',dias[i])
      
      dia = int(input("Selecciona un día para el turno (ingresa el numero correspondiente a la opcion"))

      diaElegido = dias[dia-1]

      horariosDisponibles = self.horario(diaElegido)
      print(horariosDisponibles)

      horarioElegido = int(input("Selecciona el horario: ingresa el numero correspondiente a la opcion"))

      horarioElegido = horariosDisponibles.loc[0, 'Horario']
      

      self.turnero.guardarTurno(self.id, paciente, diaElegido, horarioElegido)

   

   def horario(self, dia):
      df = self.turnero.leerDatos()
      df = df[df['id_espe'].eq(self.id)]
      df = df[['Horario']][df[dia] == 'disponible']
      return df.reset_index(drop=True)



class Paciente:
   def __init__(self):
      self.datosPacientes=[]

   def datosP(self):
      print ("Le vamos a pedir los datos del paciente")

      while True:            
  
         try:                        
            self.nombre=input("Nombre: ")
            self.nombre=str(self.nombre)
            self.apellido=input("Apellido: ")
            self.apellido=str(self.apellido)

            self.nombreCompleto = self.nombre + self.apellido
        
            edad=input("Edad: ")
            edad=int(edad)

            genero=input("Género: \n 1 Femenino (marque 1) \n 2 Masculino (marque 2) \n 3 Otro (marque 3) \n")
            genero=int(genero)
            if genero <1 or genero >3:
               print('Marque 1, 2 ó 3 según el menú anterior:')
               genero=input("Género: \n 1 Femenino (marque 1) \n 2 Masculino (marque 2) \n 3 Otro (marque 3) \n")

            dni=input("DNI: ")
            dni=int(dni)
                
            cel=input("Teléfono móvil: ")
            cel=int(cel)

            break
            
         except (ValueError): 
            print("Error al ingresar los datos")        

      self.datosPacientes.append({'Nombre':self.nombre,'Apellido':self.apellido,'Edad':edad,'Género':genero,'DNI':dni,'Celular':cel})
      self.guardarDF
      self.guardarCSV
      
      def guardarDF(self):
         dfPacientes = pd.DataFrame(self.datosPacientes)
         return dfPacientes
   
    
      def guardarCSV(self):
         self.guardarDF.to_csv('datosPacientes.csv')



odonto = Especialidad('Odontología', 1, 'Luciana Altamirano', '01:00')
oftalmo = Especialidad('Oftalmología', 2, 'Roberto Garcia', '00:20')
gineco = Especialidad('Ginecología', 3, 'Pedro Menuan', '00:30')
clinico = Especialidad('Médico Clínico', 4, 'Alejandra Ilgaramo', '00:25')


turnero = Agenda('22 al 26 de noviembre')

especialidades = [odonto, oftalmo, gineco, clinico,]
for especialidad in especialidades:
   especialidad.turnero = turnero
   especialidad.establecerTurnos(11,3,7)

pediatra = Especialidad('Pediatría', 5, 'Analia Montalvan', '00:40')
pediatra.turnero = turnero
pediatra.establecerTurnos(11,3,[1,3,5])

especialidades.append(pediatra)
#pediatra.disponibles()
#odonto.disponibles()



print("""
 Bienvenido al TURNERO del Centro de Salud Comunitario.
   La atención es de lunes a viernes de 10 a 17 hs.
""")
paciente = Paciente()
paciente.datosP()
paciente.datosPacientes
#paciente.guardarDF()

print("""
¿Para cuál especialidad quiere pedir un turno?
""")

for i in range(len(especialidades)):
   print(i+1,'-',especialidades[i].nombre,'( marque',i+1,')')

espe_elegida=input('Marque el número de la especialidad: ')
espe_elegida=int(espe_elegida)-1

print(espe_elegida)

print("""
---------------------------------------------------------

La semana habilitada es del 22 al 26 de noviembre de 2021
¿Para cuál día desea consultar los horarios disponibles?       
""")

especialidades[espe_elegida].asignarTurno(Paciente.nombreCompleto)
#diaElegido = input('Elija el día de su turno (del 1 al 5): ')
 



 Bienvenido al TURNERO del Centro de Salud Comunitario.
   La atención es de lunes a viernes de 10 a 17 hs.

Le vamos a pedir los datos del paciente


AttributeError: 'Paciente' object has no attribute 'guardarDF'